In [1]:
# imports
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain_cohere import CohereRerank
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# these three lines swap the stdlib sqlite3 lib with the pysqlite3 package
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
from langchain_chroma import Chroma
import os

In [ ]:
os.environ["OPENAI_API_KEY"] = ""
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")
llm = ChatOpenAI(model_name= "gpt-3.5-turbo", max_tokens= 500)

In [3]:
def getRelevantChunksRetriever():
    """Load PDF, split into chunks, save them to the vectordb, and return relevant chunks retriever"""
    pdf_link = "os-sertoes.pdf"
    loader = PyPDFLoader(pdf_link, extract_images=False)
    pages = loader.load_and_split()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 4000, # each chunk will have 4000 characters
        chunk_overlap = 20, # they will have an overlap of 20 characters so we do not lose much content at the end of a given chunk
        length_function = len,
        add_start_index = True
    )
    
    # split PDF into chunks
    chunks = text_splitter.split_documents(pages)

    # persist chunks into the VectorDB using OPENAI embedding model to the simple_rag_db dir
    # doing in batches to avoid exceeding token limit
    #Chroma.from_documents(chunks, embedding=embeddings_model, persist_directory="simple_rag_db")
    max_batch_size = 100
    for i in range(0, len(chunks), max_batch_size):
        batch = chunks[i:i + max_batch_size]
        Chroma.from_documents(batch, embedding=embeddings_model, persist_directory="simple_rag_db")
    
    # load vectordb
    vectordb = Chroma(embedding_function=embeddings_model, persist_directory="simple_rag_db")

    # Load Retriever - function to retrieve relevant chunks
    # k = 10 - top 10 relevant chunks (since they'll be re-ranked, it's better to get a few more relevant chunks)
    retriever = vectordb.as_retriever(search_kwargs={"k": 10}, max_tokens_limit=10000)
    return retriever

In [ ]:
os.environ["COHERE_API_KEY"] = "" # get from cohere.com/rerank

# this will use the 10 docs returned from the naive retriever to rerank and return 3 documents
rerank = CohereRerank(top_n=3, model='rerank-multilingual-v3.0')
compressor_retriever = ContextualCompressionRetriever(
    base_compressor=rerank,
    base_retriever=getRelevantChunksRetriever()
)

def ask(question):
    TEMPLATE = """
        Você é um especialista nas obras de Euclides da cunha, em especial, na obra "Os sertões". 
        Com base no contexto, responda a pergunta indicada.
        Contexto: {context}
        Pergunta: {question}
    """
    rag_prompt = ChatPromptTemplate.from_template(TEMPLATE)
    setup_retrieval = RunnableParallel({"question": RunnablePassthrough(), "context": compressor_retriever})
    output_parser = StrOutputParser()
    compressor_retrieval_chain = setup_retrieval | rag_prompt | llm | output_parser
    response = compressor_retrieval_chain.invoke(question)
    return response

In [6]:
response = ask("1. Qual é a visão de Euclides da Cunha sobre o ambiente natural do sertão nordestino e como ele influencia a vida dos habitantes?")
response

'Euclides da Cunha tinha uma visão de que o ambiente natural do sertão nordestino era extremamente árido e desafiador, caracterizado por longos períodos de seca e escassez de recursos hídricos. Ele acreditava que essa dureza do ambiente influenciava diretamente a vida dos habitantes, causando um verdadeiro martírio. Para ele, a seca e a aridez do sertão refletiam uma tortura maior, que afetava não apenas a sobrevivência física das pessoas, mas também a economia geral da vida na região. Euclides da Cunha destacava a importância da luta contra o deserto e a sede como elementos fundamentais para a sobrevivência no sertão nordestino.'

In [7]:
response2 = ask("2. Quais são as principais características da população sertaneja descritas por Euclides da Cunha? Como ele relaciona essas características com o ambiente em que vivem?")
response2

'Euclides da Cunha descreve as principais características da população sertaneja como instabilidade de complexos de fatores múltiplos e diversamente combinados, vicissitudes históricas e situação mental deplorável. Ele relaciona essas características com o ambiente em que vivem, destacando a influência do meio físico dos sertões, que apresenta uma flora agressiva, clima impiedoso, secas periódicas, solo estéril, serranias desnudas e região ingrata, tornando o local um ambiente desafiador para a vida humana. Essas condições adversas contribuem para a formação da fisionomia e características únicas da população sertaneja, que se desenvolveu fora do influxo de outros elementos e adquiriu uma fisionomia original, como se tivessem sido criados num país diverso.'

In [8]:
response3 = ask("3. Qual foi o contexto histórico e político que levou à Guerra de Canudos, segundo Euclides da Cunha?")
response3

'Segundo Euclides da Cunha, o contexto histórico e político que levou à Guerra de Canudos foi marcado por uma série de expedições militares enviadas para destruir a comunidade de Canudos, liderada pelo beato Antônio Conselheiro. As expedições foram justificadas sob a alegação de que Canudos representava uma reação monárquica contra as instituições republicanas. Além disso, houve a presença de figuras sanguinárias como o Coronel Antônio Moreira César, que serviu aos projetos políticos do governo de Floriano Peixoto. Euclides da Cunha destacou que a guerra representou um refluxo na história do Brasil, trazendo à tona uma sociedade antiga galvanizada por um líder messiânico.'

In [9]:
response4 = ask("4. Como Euclides da Cunha descreve a figura de Antônio Conselheiro e seu papel na Guerra de Canudos?")
response4

'Euclides da Cunha descreve Antônio Conselheiro como um pietista ansiando pelo reino de Deus, prometido, mas sempre delongado e ao cabo de todo esquecido pela Igreja ortodoxa do século II. Ele ordenava penitências que beneficiavam as localidades, como a reconstrução de templos abatidos, renovação de cemitérios em abandono e construção de novas edificações. Conselheiro era tolerante com as expedições sugeridas por fiéis abastados com intenções políticas, que partiam de Canudos para reforçar a soberania popular. No arraial, ele exigia ordem inalterável e mantinha os inválidos, mulheres, crianças, velhos e doentes, sob sua proteção. Ele era considerado um Santo protetor pelos seus seguidores, que o saudavam com versos correntes nos sertões.'

In [10]:
response5 = ask('5. Quais são os principais aspectos da crítica social e política presentes em "Os Sertões"? Como esses aspectos refletem a visão do autor sobre o Brasil da época?')
response5

'Os principais aspectos da crítica social e política presentes em "Os Sertões" estão relacionados à desigualdade social, à violência, à miséria e à falta de desenvolvimento do sertão nordestino. Euclides da Cunha critica a situação precária em que viviam os sertanejos, explorados e abandonados pelo poder público, além de denunciar a brutalidade das forças militares na guerra de Canudos.\n\nEsses aspectos refletem a visão do autor sobre o Brasil da época como um país marcado pela desigualdade social, pela violência e pela falta de integração entre as diferentes regiões. Euclides da Cunha mostra como a sociedade brasileira estava dividida entre o litoral desenvolvido e o sertão atrasado, evidenciando a necessidade de uma maior atenção do Estado para com as populações marginalizadas. Além disso, o autor critica a mentalidade colonialista e eurocêntrica que predominava entre as elites brasileiras, que viam o sertanejo como um ser inferior e bárbaro em comparação com a civilização litorânea